In [1]:
# Librerías
from prody import *
import csv
from prody import * 
from Bio import pairwise2, SeqIO
import os
import glob

In [2]:
# Nombre de la proteina de interes
prot_name = 'fxa'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P00742"

In [3]:
# Secuencia de la CDK2 de UniProt
fasta = SeqIO.read(open(F'../data/{uniprot_id}.fasta'),'fasta')
seq_prot = str(fasta.seq)
print(seq_prot)

MGRPLHLVLLSASLAGLLLLGESLFIRREQANNILARVTRANSFLEEMKKGHLERECMEETCSYEEAREVFEDSDKTNEFWNKYKDGDQCETSPCQNQGKCKDGLGEYTCTCLEGFEGKNCELFTRKLCSLDNGDCDQFCHEEQNSVVCSCARGYTLADNGKACIPTGPYPCGKQTLERRKRSVAQATSSSGEAPDSITWKPYDAADLDPTENPFDLLDFNQTQPERGDNNLTRIVGGQECKDGECPWQALLINEENEGFCGGTILSEFYILTAAHCLYQAKRFKVRVGDRNTEQEEGGEAVHEVEVVIKHNRFTKETYDFDIAVLRLKTPITFRMNVAPACLPERDWAESTLMTQKTGIVSGFGRTHEKGRQSTRLKMLEVPYVDRNSCKLSSSFIITQNMFCAGYDTKQEDACQGDSGGPHVTRFKDTYFVTGIVSWGEGCARKGKYGIYTKVTAFLKWIDRSMKTRGLPKAKSHAPEVITSSPLK


In [4]:
import sys
sys.path.append(r'../.')
from modules.find_gaps import find_gaps
# Recibe una secuencia (con gaps como "-") y un valor r para valorar el gap en +/- r posiciones 

In [5]:
# Los IDs de los cristales están en el archivo
import pandas as pd
import glob
file_path = glob.glob(F"../data/TABLA_MTDATA_{prot_name.upper()}_*.json")[0]
df_prot = pd.read_json(file_path)
pdbids_list = df_prot.set_index('PDB_ID').index
print("Total de PDB IDs a modelar:", len(pdbids_list))

Total de PDB IDs a modelar: 136


In [6]:
# Directiros y parámetros base
struct_dir = F'../../FILES/CRYSTALS/PROT_{prot_name.upper()}_CHAINS/'
tail_pdb = '_A.pdb' # extensión del archivo pdb del cristal
tail_model = '_MODLL' # epiteto del archivo del modelo final, la extensión .pdb la agrega modeller
pdbs_model_dir = F"../../FILES/CRYSTALS/PROT_{prot_name.upper()}_MODELOS_modeller_NoPrep/" # Carpeta de salida para los modelos

# Creamos el directorio de salida si es que no existe
import os
if not os.path.exists(pdbs_model_dir):
    os.makedirs(pdbs_model_dir)

In [7]:
# FIND GAP FUNCTION WAS CORRECTED: find_gaps

In [8]:
from modeller import * # licencia modeller necesaria
import shutil
# vi /home/joel/anaconda3/envs/mds/lib/modeller-9.22/modlib/modeller/config.py
# MODELIRANJE
from modeller.automodel import *


def run_modeller(pdb_code, seq_prot, num_res_window = 0, force_modeling = False,
                max_var_iterations = 500, repeat_optimization = 2):
    '''función para modelar gaps en estructuras cristalográficas utilizando Modeller'''
    ###########################
    if not force_modeling:
        if os.path.isfile(pdbs_model_dir + pdb_code + tail_model + ".pdb"):
            return print("El modelo ya existe en el directorio")
    ###########################
    try:
        # Se lee el archivo pdb. La selección omite residuos negativos.
        # Además omite los residuos no estandar, o con modificaciones postraduccionales como el TPO (Tirosina fosfatada)
        stc_prot = parsePDB(struct_dir 
                            + pdb_code + tail_pdb).select('not nonstdaa and resid > 0') 
        chaid = stc_prot.getChids()[0] # Obtiene el id de la cadena
        ref_hv = stc_prot.getHierView()[chaid] # Se obtiene sólo la cadena
        seq_cry = ref_hv.getSequence() # Se obtiene la secuencia de la estructura cristalográfica 
        print(seq_cry)
    except:
        return print("Error al abrir y modelar:", pdb_code)
    ########################### 
    # Pregunta si la longitud y la secuencias de Uniprot y de la estructura son iguales
    same_seq = len(seq_cry) == len(seq_prot) and seq_cry == seq_prot
    
    # Nombre de los cabezales de las secuencia deben coincidir con el de los archivos de entrada y salida
    crys_file_name = pdb_code + tail_pdb
    model_file_name = pdb_code + tail_model
    
    # si same_seq es verdadero, se omite hacer el modelado pues la estructura está completa
    if same_seq:
        shutil.copy(struct_dir + pdb_code + tail_pdb, 
                    pdbs_model_dir + pdb_code + tail_model + ".pdb")
        return print("La proteína " + pdb_code + " ya está completa") 
        # Si le hacen falta átomos a un residuo fuera del gap, esto se corregirá 
        # con PDB2PQR y pdb4amber en un fase posterior al modelado.
    ###########################     
    print("Modelando proteína " + pdb_code)
    # Obtener la secueNcia de la estructura a modelar y guardarla en un archivo.
    # Alineamiento global, se penalizan con -10 la abertura de nuevos gaps, se obtiene el mejor alineamiento.
    alignment = pairwise2.align.globalxs(seq_cry, seq_prot, -10, -0.5, gap_char='-')[0]
    # Secuencias del alineamiento
    algn1_struc = alignment[0] # Secuencia alineada de la estructura cristalográfica
    algn2_seq = seq_prot # Secuencia completa de UniProt

    

    ''' NECESARIO: There should be 10 fields separated by colons ":".
    Please check the file to make sure your sequences end with the '*' character.
    Nomenclaturas de los campos del header: 
    https://salilab.org/modeller/8v2/manual/node176.html'''

    # HEADERS (la secuencia del cristal va primero, luego la secuencia completa)
    struc_header = "structureX:" + crys_file_name + ":.:" + chaid + ":.:" + chaid + ":.:.:.:"
    seq_header = "sequence:" + model_file_name + ":.:.:.:.:.:.:.:"
    
    # Crea el archivo de alineamiento con la estructura requerida por modeller
    alg_filename = pdb_code + ".alg"
    with open(alg_filename, "w") as handle:
        handle.write("\n>P1;%s\n%s\n%s*\n>P1;%s\n%s\n%s*\n" % (crys_file_name, struc_header, algn1_struc, 
                                                               model_file_name, seq_header, algn2_seq))
    ##########################
    # Ejecuta find_gaps para obtener los posibles gaps de la secuencia
    gaps = find_gaps(algn1_struc, r = num_res_window)
    num_gaps = gaps["num_gaps"]
    gap_i = gaps["gap_window"]
    print(gaps)
    #gap_i = [[gap - 1 for gap in gap_i[0]]]
    
    
    # Se crea el string que necesita Modeller para definir al primer gap
    s = "self.residue_range('" + str(gap_i[0][0]) + "', '" + str(gap_i[0][1]) + "')"

    # A pesar de que sólo haya un gap, se ejecuta el ciclo, 
    # si hay más de uno, se extiende el String para incluir los demás
    for i in range(1, num_gaps):
        s = s + ", " + "self.residue_range('" + str(gap_i[i][0]) + "', '" + str(gap_i[i][1]) + "')" #nótese la coma
    ##########################
    ''' SE EJECUTA MODELLER'''
    ##########################
    # Se crea un nuvo ambiente de modeller
    env = environ()
    env.io.atom_files_directory = ['.', struct_dir]
    
    # Se modifica la clase MyModel de Modeller para fijar los residuos ya existentes en el cristal
    # Estos residuos no se modelarán ni sus átomos cambiarán de posición
    # NECESARIO para poder capturar los valores obtenidos por el rango de gaps
    
    MyModel_code = """
class MyModel(automodel):
    def select_atoms(self):
        return selection(""" + s + """)

    def special_patches(self, aln):

        self.patch(residue_type='DISU', residues=(self.residues['7:'], self.residues['12:']))
        self.patch(residue_type='DISU', residues=(self.residues['27:'], self.residues['43:']))
        self.patch(residue_type='DISU', residues=(self.residues['156:'], self.residues['170:']))
        self.patch(residue_type='DISU', residues=(self.residues['181:'], self.residues['209:']))

        
""" # Al estar dentro del loop la identación de este string es importante
    exec(MyModel_code, globals()) # Se lleva a cabo la modificación a MyModel para agregar la región fijada
    # Se intancia el objeto MyModel
    a = MyModel(env, alnfile = alg_filename, # Lee el archivo fasta creado y guardado en el directorio actual
                      knowns = crys_file_name, # Archivo pdb crys, que coincide con el id en el archivo fasta
                      sequence = model_file_name) # Nombre del modelo
    a.starting_model= 1
    a.ending_model  = 1
    # Se definen los parámetros de refinamiento dle modelo:
    # Más info sobre el refinamiento: https://salilab.org/modeller/9.21/manual/node19.html
    a.library_schedule = autosched.slow # Originalmente comentado
    a.max_var_iterations = max_var_iterations #*500
    a.md_level = refine.slow # Nivel del refinamiento
    
    a.repeat_optimization = repeat_optimization # Número de repeticiones para la optimización
    
    a.make()

    ###########################
    # Renombra el archivo pdb a pdb_code + _full.pdb
    # DEBE ser el único archivo pdb en el directorio '.'
    model_file = glob.glob('./' + pdb_code + '*.pdb')[0] # Nombre del archivo pdb
    
    os.rename(model_file, pdbs_model_dir + pdb_code + tail_model + ".pdb") # Mueve el pdb a la carpeta model_pdbs
    # Elimina los archivos extra generados - Todos empiezan con el codigo del PDB ID
    for f in glob.glob(pdb_code + "*"):
        os.remove(f)

In [9]:
seq_prot = 'IVGGQECKDGECPWQALLINEENEGFCGGTILSEFYILTAAHCLYQAKRFKVRVGDRNTEQEEGGEAVHEVEVVIKHNRFTKETYDFDIAVLRLKTPITFRMNVAPACLPERDWAESTLMTQKTGIVSGFGRTHEKGRQSTRLKMLEVPYVDRNSCKLSSSFIITQNMFCAGYDTKQEDACQGDSGGPHVTRFKDTYFVTGIVSWGEGCARKGKYGIYTKVTAFLKWIDRSMKTRGLPKAKS'

In [10]:
len(seq_prot)

242

In [12]:
import time
start = time.time()

# se lleva a cabo el modelado de todas las estructuras cristalográficas
for pdb_code in pdbids_list:
    run_modeller(pdb_code,
                 seq_prot,
                 num_res_window = 0, 
                 force_modeling = False,
                 max_var_iterations = 1000, 
                 repeat_optimization = 2)

end = time. time()
print(end - start)

@> 1837 atoms and 1 coordinate set(s) were parsed in 0.01s.


El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
El modelo ya existe en el directorio
E

@> 1850 atoms and 1 coordinate set(s) were parsed in 0.01s.


iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_484W> Dihedral still outside +-90:      -91.4516
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_484W> Dihedral still outside +-90:       90.6228
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_484W> Dihedral still outside +-90:      -91.3886
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      242
Number of all, selected real atoms                :     1912      73
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :    24496     281
COVALENT_CYS              


check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
patch_s_522_> Number of disulfides patched in MODEL:        4
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.

...

iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_484W> Dihedral still outside +-90:      -90.7997
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      242
Number of all, selected real atoms                :     1912      60
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :    24662     228
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):       63
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390


check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
patch_s_522_> Number of disulfides patched in MODEL:        4
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.

@> 1831 atoms and 1 coordinate set(s) were parsed in 0.01s.


iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_484W> Dihedral still outside +-90:      -93.4807
iupac_m_397W> Atoms were not swapped because of the uncertainty of how to handle the H atom.
iupac_m_484W> Dihedral still outside +-90:      -90.7796


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      242
Number of all, selected real atoms                :     1912      80
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :    24607     308
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):      127
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH               


check_ali___> Checking the sequence-structure alignment. 

Implied intrachain target CA(i)-CA(i+1) distances longer than  8.0 angstroms:

ALN_POS  TMPL  RID1  RID2  NAM1  NAM2     DIST
----------------------------------------------
END OF TABLE
read_to_681_> topology.submodel read from topology file:        3
patch_s_522_> Number of disulfides patched in MODEL:        4
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
patch___481W> This disulfide was already patched.
mdtrsr__446W> A potential that relies on one protein is used, yet you have at
              least one known structure available. MDT, not library, potential is used.
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies
condens_443_> Restraints marked for deletion were removed.